# Starting / Stopping the Cluster

In [2]:
zone=us-west1-b
cluster=tmb-cluster
cpu_nodes=6
cpu_cores=32
cpu_machine="n1-standard-$cpu_cores"
scopes="default,storage-rw,compute-ro"
maxpods=$((2 * $cpu_cores))
netsize=10

cluster_flags="--enable-basic-auth --no-enable-autoupgrade"
cluster_flags="--enable-ip-alias --cluster-ipv4-cidr 10.0.0.0/$netsize $cluster_flags"
cluster_flags="--max-pods-per-node=$maxpods $cluster_flags"

account=$(gcloud auth list --filter=status:ACTIVE --format='value(account)')
echo "active account: $account"
env | egrep 'GCE_PROJECT|GOOGLE_'

active account: tbreuel@nvidia.com
GOOGLE_APPLICATION_CREDENTIALS=/home/tmb/.KEYS/Research-1c395429d090.json
GCE_PROJECT=research-191823


In [3]:
yes | gcloud container clusters delete ${cluster}

The following clusters will be deleted.
 - [tmb-cluster] in [us-west1-b]

Do you want to continue (Y/n)?  
Deleting cluster tmb-cluster...
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [4]:
if gcloud container clusters get-credentials ${cluster}; then
    echo ${cluster} already exists
else
    gcloud config set project ${project}
    gcloud config set compute/zone ${zone}
    gcloud config set container/new_scopes_behavior true || true
    #gcloud config set container/use_v1_api true
    gcloud config set container/cluster ${cluster}
    gcloud beta container clusters create ${cluster} \
        ${cluster_flags} \
        --zone ${zone} \
        --machine-type ${cpu_machine} \
        --scopes ${scopes} \
        --num-nodes ${cpu_nodes} # --cluster-version 1.9
fi

Fetching cluster endpoint and auth data.
kubeconfig entry generated for tmb-cluster.
tmb-cluster already exists


In [10]:
flags=--no-cache
tee /dev/stderr <<EOF |
FROM ubuntu:18.04
ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get -qqy update 

RUN apt-get install -qqy curl gcc openssh-client git
RUN apt-get install -qqy iputils-ping dnsutils parallel dtrx
RUN apt-get install -qqy apt-transport-https lsb-release gnupg apt-utils

ENV TZ="America/Los Angeles"
RUN echo \$TZ > /etc/timezone
RUN rm -f /etc/localtime && ln -snf /usr/share/zoneinfo/$TZ /etc/localtime || true
RUN apt-get install -qqy tzdata

RUN apt-get install -qqy python-dev python-setuptools python-pip
RUN curl https://dl.google.com/dl/cloudsdk/release/google-cloud-sdk.tar.gz > /tmp/google-cloud-sdk.tar.gz
RUN mkdir -p /usr/local/gcloud && tar -C /usr/local/gcloud -xvf /tmp/google-cloud-sdk.tar.gz
RUN /usr/local/gcloud/google-cloud-sdk/install.sh
ENV PATH \$PATH:/usr/local/gcloud/google-cloud-sdk/bin

RUN apt-get install -qqy python3 python3-pip
RUN apt-get install -qqy python3-matplotlib
RUN pip3 install simplejson braceexpand
RUN apt-get install -qqy libmpack-dev
RUN pip3 install msgpack
RUN apt-get install -qqy python3-zmq
RUN apt-get install -qqy net-tools
RUN apt-get install -qqy expect
RUN apt-get install -qqy python3-skimage
RUN apt-get install -qqy python3-sklearn
RUN pip3 install beautifulsoup4
RUN pip3 install lxml
RUN pip3 install future
RUN pip3 install six
RUN pip3 install torch torchvision

RUN echo $(date)
RUN pip3 install https://github.com/tmbdev/tarproc/archive/master.zip
RUN pip3 install https://github.com/tmbdev/webdataset/archive/master.zip

RUN mkdir /work
WORKDIR /work
EOF
docker build $flags -t simple - &&
docker tag simple gcr.io/research-191823/simple &&
# run: gcloud auth configure-docker
docker push gcr.io/research-191823/simple

FROM ubuntu:18.04
ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get -qqy update 

RUN apt-get install -qqy curl gcc openssh-client git
RUN apt-get install -qqy iputils-ping dnsutils parallel dtrx
RUN apt-get install -qqy apt-transport-https lsb-release gnupg apt-utils

ENV TZ="America/Los Angeles"
RUN echo $TZ > /etc/timezone
RUN rm -f /etc/localtime && ln -snf /usr/share/zoneinfo/ /etc/localtime || true
RUN apt-get install -qqy tzdata

RUN apt-get install -qqy python-dev python-setuptools python-pip
RUN curl https://dl.google.com/dl/cloudsdk/release/google-cloud-sdk.tar.gz > /tmp/google-cloud-sdk.tar.gz
RUN mkdir -p /usr/local/gcloud && tar -C /usr/local/gcloud -xvf /tmp/google-cloud-sdk.tar.gz
RUN /usr/local/gcloud/google-cloud-sdk/install.sh
ENV PATH $PATH:/usr/local/gcloud/google-cloud-sdk/bin

RUN apt-get install -qqy python3 python3-pip
RUN apt-get install -qqy python3-matplotlib
RUN pip3 install simplejson braceexpand
RUN apt-get install -qqy libmpack-dev
RUN pip3 install msgpack


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



google-cloud-sdk/lib/googlecloudsdk/third_party/apis/container/v1beta1/container_v1beta1_messages.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/container/v1beta1/resources.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/container_v1.json
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/container_v1alpha1.json
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/container_v1beta1.json
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/__init__.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/v1/
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/v1/__init__.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/v1/containeranalysis_v1_client.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containeranalysis/v1/containeranalysis_v1_messages.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/containera

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 ---> Running in ae68e2a99435
Welcome to the Google Cloud SDK!
the installation to be inaccessible to users other than the root user.
Do you want to help improve the Google Cloud SDK (y/N)?  


Your current Cloud SDK version is: 271.0.0
Installing components from version: 271.0.0

+----------------------------------------------------------------------------+
|                    These components will be installed.                     |
+-----------------------------------------------------+------------+---------+
|                         Name                        |  Version   |   Size  |
+-----------------------------------------------------+------------+---------+
| BigQuery Command Line Tool                          |     2.0.50 | < 1 MiB |
| BigQuery Command Line Tool (Platform Specific)      |     2.0.50 | < 1 MiB |
| Cloud SDK Core Libraries (Platform Specific)        | 2019.11.08 | < 1 MiB |
| Cloud Storage Command Line Tool                     |       4.46 | 3.6 MiB |
| Cloud

In [9]:
date

Sat Nov 16 21:31:57 PST 2019


In [13]:
mkdir -p scripts
cp hocr2seg scripts/.
cp seg2patches scripts/.
cp $GOOGLE_APPLICATION_CREDENTIALS scripts/google_application_credentials.json
kubectl delete configmap files || true
(
cd scripts
args=
for f in *; do args="$args --from-file=$f=$f"; done
echo -- $args
kubectl create configmap files $args
)

configmap "files" deleted
/home/tmb/exp/ocr-prep/scripts
-- --from-file=google_application_credentials.json=google_application_credentials.json --from-file=hocr2markers=hocr2markers --from-file=hocr2seg=hocr2seg --from-file=key.json=key.json --from-file=seg2patches=seg2patches --from-file=shufflebatch=shufflebatch
configmap/files created
